### Importações necessárias

In [1]:
import numpy as ny
import math
from numpy import *
from numpy.linalg import inv
import numpy.linalg as la
import csv

### Função de impressão tester

In [2]:
def pmatriz(matrizz, mens):
    matrizz = ny.array(matrizz)
    print('*=' * 50)
    print(f'{mens:^90}')
    print('*=' * 50)
    print('      ', end='')
    for i in range(1, matrizz.shape[1] + 1):
        print(f'{i:^7}', end='')
    print('\n', '*=' * 50)
    for i in range(matrizz.shape[0]):
        print(f'{i + 1:>2} == ', end='')
        for j in range(matrizz.shape[1]):
            if type(matrizz[i][j]) == ny.int32:
                print(f'{matrizz[i][j]:>4} ', end='')
            else:
                print(f'{matrizz[i][j]:>6.3f} ', end='')
        print()
    print('*=' * 50)

### Função para cálculo dos suportes dos pontos médios dos itens de determinada Regra de Associação

In [3]:
def calc_sup(linha_da_RA, vetor_dos_itens, Matriz_P_de_Respostas):
    item = vetor_dos_itens[linha_da_RA]
    soma = 0
    for i in range(Matriz_P_de_Respostas.shape[0]):
        verdade = True
        for j in item:
            if Matriz_P_de_Respostas[i,j-1] != 1:
                verdade = False
        if verdade:
            soma += 1
    return soma

### Função para cálculo do ponto médio

In [4]:
def calcpm(ptos, matriz_de_coordenadas_dos_itens):
    if len(ptos) == 1:
        ptm = matriz_de_coordenadas_dos_itens[ptos[0] - 1]
    else:
        soma = 0
        for j in ptos:
            soma += matriz_de_coordenadas_dos_itens[j - 1]
        ptm = soma / len(ptos)
    return ptm

### Função para cálculo da distância entre os pontos médios antecedente e consequente

In [5]:
def calcdistfim(roh, px1, px2, sup1, sup2, tam, ditens):
    for k in range(len(px1)):
        a = px1[k]
        b = px2[k]
        c = math.sqrt(sup1/tam)
        d = math.sqrt(sup2/tam)
        ditens += roh[k] * ((a/c 
                            - b/d)**2) #Qui-quadrado
    #print(ditens)
    return ditens

### Função para cálculo da distância entre itens e submatrizes de distâncias dada det. regra de associação.

In [6]:
def dist2ptsi(ptos, M_cides, V_fc, tamanho, roh, n_lin):
    Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    i=0
    while i in range(len(ptos)):
        for j in range(len(ptos)):
            d_itens = 0
            linha1 = ptos[i]-1
            linha2 = ptos[j]-1
            for k in range(tamanho):
                d_itens += roh[k] * (((M_cides[linha1][k])/math.sqrt(V_fc[linha1]/n_lin) 
                            - (M_cides[linha2][k])/math.sqrt(V_fc[linha2]/n_lin))**2) #Qui-quadrado
            if i != j:
                Ditens[i,j] = round(d_itens,2)
                Ditens[j,i] = round(d_itens,2)
        i+=1
    return Ditens

### Função para cálculo da distância entre transações

In [7]:
def dist2ptst(ptos):
    ematriz = (len(ptos) == Mctd.shape[0])
    if ematriz:
        Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    if len(ptos) < 2:
        return 'Hove um erro e foi passado apenas um ponto!'
    i=0
    while len(ptos) >= 2:
        while i in range(len(ptos)):
            for j in range(1,len(ptos)):
                d_itens = 0
                linha = ptos[0]-1
                coluna = ptos[j]-1
                for k in range(Des):
                    d_itens += rho[k] * (((Mctd[linha][k]/math.sqrt(Vfl[linha]/ncol))
                                - (Mctd[coluna][k]/math.sqrt(Vfl[coluna]/ncol)))**2) #Qui-quadrado
                if ematriz:
                    Ditens[linha,coluna] = round(d_itens,2)
                    Ditens[coluna,linha] = round(d_itens,2)
                else: print(f'Distância entre {ptos[i]} e {ptos[j]} = {d_itens:.2f}')
            ptos.pop(0)
    if ematriz: return Ditens

### Criando a matriz[n][m] de respostas a partir dos dados fornecidos pelo mestre Altobelli.

In [8]:
arq = open('bloodPressureNishisatobook-base.D18.N15.txt', 'r')
texto = arq.readlines()
matriz = []
lres = []
nrpp = 3  # número de opções por perguntas feitas para cada transação
for linha in texto:
    limite = 0
    linElem = linha.split()
    while limite < len(linElem):
        a = int(linElem[limite]) - nrpp * limite
        for resp in range(nrpp):
            if (resp+1)/a == 1:
                lres += [1]
            else:
                lres += [0]
        limite += 1
    matriz += [lres]
    lres = []
arq.close()
MPR = ny.matrix(matriz)
# print(*matrizF,sep='\n')
ncol = MPR.shape[1]
nlin = MPR.shape[0]
nperguntas = ncol // nrpp
Des = ncol - nperguntas - 1
# print('\nNúmero de colunas = ',ncol)
# print('Número de linhas = ',nlin)
# print('Número de perguntas = ',nperguntas)
# print('Dimensões do espaço solução = ',Des)
# pmatriz(MPR,'Matriz de Padrão de Respostas')

### Matriz transposta da MPR

In [9]:
MPRt = ny.transpose(MPR)
#pmatriz(MPRt,'Matriz Transposta de Padrão de Respostas')

### Vetor de frequência de linhas (Vfl)

In [10]:
#Cálculo da Matriz Diagonal de Linhas de F = Vfl
Vfl = ny.sum(MPR, axis=1)
#print (Vfl)

### Vetor de frequência de colunas (Vfcc)

In [11]:
Vfcc = []
soma = 0
for i in range(ncol):
    for j in range(len(matriz)):
        soma += matriz[j][i]
    Vfcc.append(soma)
    soma = 0
Vfc = ny.sum(MPR, axis=0)
# print(Vfc.shape,'\n')
# print(len(Vfcc))
print(Vfc,'\n\n',Vfcc)

[[ 5  6  4  3  3  9  4  5  6  2  3 10  9  3  3  5  5  5]] 

 [5, 6, 4, 3, 3, 9, 4, 5, 6, 2, 3, 10, 9, 3, 3, 5, 5, 5]


### Cálculo das matrizes diagonais - MdL = matriz diagonal de linhas(nxn) - MdC = matriz diagonal de colunas(mxm)

In [12]:
MdC = ny.diagflat(Vfc)
MdL = ny.diagflat(Vfl)
# print('   Matriz Diagonal de Linhas')
# print(*MdL, sep='\n')
# print('     Matriz Diagonal de Colunas')
# print(*MdC, sep = '\n')

### Matriz de correlação entre colunas

In [13]:
MdCi = inv(MdC)
MdLi = inv(MdL)
Mcc = MPRt * MdLi * MPR * MdCi
# pmatriz(Mcc,'Matriz de Correlação entre Colunas')

### Vetor de autovalores(Vetaval) e matriz de autovetores(Mavet) com normalização

In [14]:
vetaval, mavet = la.eig(Mcc)
Vetaval = ny.flip(ny.sort(vetaval, axis=0), 0)
valaux = mavet.T.tolist()  # transforma uma matriz numpy em uma lista para facilitar a manipulação dos dados.
Mavet = []  # Cria a Matriz de autovetores ordenada em função dos autovalores
for elem in Vetaval:
    ix = vetaval.tolist().index(elem)
    Mavet.append(valaux[ix])
Mavet = ny.array(Mavet).T.astype(ny.float32)  # converte números complexos para float 32
Vetaval = Vetaval.astype(ny.float32)
# '\n',Vetaval)
Vetaval = ny.delete(Vetaval, 0, 0)  # apaga a solução trivial no vetor de autovalores e na matriz
Mavet = ny.delete(Mavet, 0, 1)  # de autovetores removendo a primeira dimensão
while Des < Vetaval.shape[0]:
    Vetaval = ny.delete(Vetaval, Des, 0)
    Mavet = ny.delete(Mavet, Des, 1)

C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  
C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


### Cálculo do Rho

In [19]:
rho = list()

for x in Vetaval:
    rho.append(sqrt(x))
    
rho = ny.array(rho)
# for i in range(len(rho)):
#   print(f'{rho[i]:>5.4f}  ', end='')

### Matriz resultante do porduto de Hadamard(MpHad)

In [20]:
MpHad = ny.multiply(Mavet, Mavet)
# print(MpHad)

### Calculando a Matriz T e seu vetor de frequência de colunas

In [21]:
matrizT = MdC * MpHad #equação(7)
matrizT = ny.array(matrizT)
VfcT=list()
VfcT=ny.sum(matrizT, axis=0) #soma coluna, caso axis=1 somaria linha = equação(8)
#for i in VfcT:
    #print(f'{i:>5.4f} ', end='')
#pmatriz(matrizT,'Matriz T')

### Vetor de multiplicadores e escalar da matriz padrão-de-respostas. eq(9)

In [23]:
escalarMPR = ny.sum(matriz)
VetMulticT = list()
for i in VfcT:
    VetMulticT.append(math.sqrt(escalarMPR / i))
# for i in range(len(VetMulticT)):
#   print(f'{VetMulticT[i]:>7.4f}', end='')

### Cálculo da Matriz de Pesos Padrão dos itens(Mppi = x_normed)

In [24]:
# Equação(10) Matriz de pesos padrão dos itens(Mppi)
Mppi = list()
aux = list()
for i in range(Mavet.shape[0]):
    for n in range(len(VetMulticT)):
        aux.append(Mavet[i][n] * VetMulticT[n])
    Mppi.append(aux[:])
    aux.clear()
Mppi = ny.array(Mppi)
# pmatriz(Mppi,'Matriz de Pesos Padrão dos Itens')

### Matriz das Coordenadas dos Itens nas Dimensões (Mcides = x_projected)

In [25]:
Mcides = Mppi * rho
# pmatriz(Mcides,'Matriz de Coordenadas dos Itens nas Dimensões')

### Matriz das Coordenadas das Transações nas Dimensões

In [26]:
Ctdes = MPR * Mppi  # fórmula (13) matriz W
Ctdes = ny.array(Ctdes)
#pmatriz(Ctdes,'Matriz de Coordenadas das Transações nas Dimensões')

### Matriz de Pesos Padrão das Transações (Mppt = y_normed)

In [27]:
Mppt = Ctdes / (Vfl * rho)  # fórmula (14)
#pmatriz(Mppt,'Matriz de Pesos Padrão de Transações')

### Matriz de Coordenadas das Transações nas Dimensões (Mctd = y_projected)

In [28]:
Mctd = Ctdes / Vfl  # fórmula (15) simplificada
Mctd = ny.array(Mctd)
# pmatriz(Mctd,'Matriz de Coordenadas das Transações de itens')

### Calculando o Delta

In [29]:
Delta = Vetaval * (100 / ny.sum(Vetaval))
for i in Delta:
    print(f'{i:5.4f}  ', end='')

27.2059  18.7362  17.2742  15.3483  6.5346  6.0108  3.7515  2.3661  1.5778  0.8652  0.3294  

### Preparando a montagem do arquivo de extensão CSV

In [30]:
arq2 = open('bloodPressureNishisatobook.D18.N15.txt', 'r') # ARQUIVO QUE CONTÉM AS REGRAS DE ASSOCIAÇÃO DE INTERESSE
texto2 = arq2.readlines()
arq2.close()
confList = []
confDesejada = 0  # 40% de confiança
for x in texto2:
    linhatxt = x.split()
    conflinha = float(linhatxt[len(linhatxt) - 1])
    if conflinha >= confDesejada:
        confList.append(linhatxt[:])
ra_antcd = []
ra_consq = []
sup = []
aux1 = []
aux2 = []
pm_antcd = []
pm_consq = []
for y in confList:
    ind = y.index('==>')
    sup.append(y[y.index('#SUP:') + 1])
    for ini in range(len(y)):
        if y[ini] == '#SUP:':
            ra_consq.append(aux2[:])
            aux2.clear()
            ini = len(y)
            break
        elif ini < ind:
            aux1.append(int(y[ini]))
            # next
        elif y[ini] == '==>':
            if len(aux1) > 1:
                ra_antcd.append(aux1[:])
            else:
                ra_antcd.append(aux1[:])
            aux1.clear()
            # next
        elif ini > ind:
            aux2.append(int(y[ini]))
for i in range(len(confList)):
    confianca = float(confList[i][len(confList[i]) - 1]) * 100
    pm_antcd.append(calcpm(ra_antcd[i], Mcides))  # pontos médios de todos antecedentes
    pm_consq.append(calcpm(ra_consq[i], Mcides))  # pontos médios de todos consequentes
pm_antcd = ny.array(pm_antcd)
pm_consq = ny.array(pm_consq)
# if len(ra_antcd[i])>= 2 and len(ra_consq[i])>= 2:
# print(f'projeção de {ra_antcd[i]} em {ra_consq[i]} suporte = {sup[i]} e confiança = {confianca:5.1f}% pm_antcd {pm_antcd[i]} pm_consq {pm_consq[i]}.')
# for y in confList:
#   a=float(y[len(y)-1])*100
#  print(f'Confiança de {a:5.1f}%')
# for i in range(len(confList)):
# print(pm_antcd[i],' e ', pm_consq[i])
# print(total)

### Apresentação da Matriz de Distância entre Itens ou outra matriz qualquer dado um conjunto de itens

In [31]:
#a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
#result = dist2ptsi(a[:], Mcides, Vfc, Des, rho, nlin)
#pmatriz(result,'Distância entre todos os itens')
print()

### Cálculo do suporte do pontos médios antecedente e consequente

In [32]:
supant = []
supcons = []
for linha_RA in range(len(ra_antcd)):
    supant.append(calc_sup(linha_RA, ra_antcd, MPR))
    supcons.append(calc_sup(linha_RA, ra_consq, MPR))
    #print('linha = ',linha_RA+1, 'sup antec.',ra_antcd[linha_RA],' = ',supant[linha_RA],' === sup conseq.',ra_consq[linha_RA],' = ',supcons[linha_RA])'''
# Cálculo do Lift
#Lift = []
#for regras in range(len(supcons)):
#    Lift.append(round(float(confList[regras][len(confList[regras])-1])/supcons[regras],4))

### Cálculo do vetor de distância final entre os pontos médios antecedentes e consequentes

In [34]:
distantcdconseq = []
origem = ny.zeros(Des)
dist_antecd_origem = []
dist_conseq_origem = []
for i in range(len(ra_antcd)):
    a = calcdistfim(rho, pm_antcd[i], pm_consq[i], supant[i], supcons[i], nlin, 0)
    distantcdconseq.append(a)
    a = calcdistfim(rho, pm_antcd[i], origem, supant[i], supcons[i], nlin, 0)
    dist_antecd_origem.append(a)
    a = calcdistfim(rho, pm_consq[i], origem, supcons[i], supant[i], nlin, 0)
    dist_conseq_origem.append(a)
for i in range(len(dist_conseq_origem)):
    print('Para ',i,' = ', dist_conseq_origem[i])
for i in range(len(dist_antecd_origem)):
    print('Para ',i,' = ', dist_antecd_origem[i])

Para  0  =  3.8088252920490215
Para  1  =  2.135999715186046
Para  2  =  3.2039995727790673
Para  3  =  4.66422408848213
Para  4  =  3.2039995727790673
Para  5  =  4.605431154431794
Para  6  =  3.2039995727790673
Para  7  =  3.5126766197066734
Para  8  =  3.2039995727790673
Para  9  =  1.7192753113320216
Para  10  =  3.2039995727790673
Para  11  =  3.0865610954988334
Para  12  =  2.135999715186046
Para  13  =  1.7192753113320216
Para  14  =  2.135999715186046
Para  15  =  2.1259683727539276
Para  16  =  2.135999715186046
Para  17  =  3.021417368817434
Para  18  =  2.135999715186046
Para  19  =  3.077366591127069
Para  20  =  4.245828857256073
Para  21  =  1.7192753113320216
Para  22  =  3.5126766197066734
Para  23  =  4.290501127225049
Para  24  =  3.5126766197066734
Para  25  =  2.1259683727539276
Para  26  =  3.620854613701271
Para  27  =  1.7192753113320216
Para  28  =  1.7192753113320216
Para  29  =  4.377593489461255
Para  30  =  1.7192753113320216
Para  31  =  3.021417368817434
P

Para  380  =  3.696108414404421
Para  381  =  2.781841690607453
Para  382  =  3.215679174554097
Para  383  =  2.228103976213526
Para  384  =  4.124811260949786
Para  385  =  2.135999715186046
Para  386  =  3.620854613701271
Para  387  =  1.7192753113320216
Para  388  =  3.021417368817434
Para  389  =  3.2751474213019165
Para  390  =  4.445924992463386
Para  391  =  3.1864655601050247
Para  392  =  5.192092127841759
Para  393  =  2.8216662932835312
Para  394  =  1.698030335923473
Para  395  =  2.8917656868737796
Para  396  =  2.6311171791858077
Para  397  =  4.0251201870271345
Para  398  =  2.4803916538860653
Para  399  =  3.403392064534286
Para  400  =  2.722629486515959
Para  401  =  4.965389857165805
Para  402  =  3.606887091728527
Para  403  =  5.980084420909995
Para  404  =  4.633742635854722
Para  405  =  2.4218468742174446
Para  406  =  3.5165484124817583
Para  407  =  3.0361307208864057
Para  408  =  3.596650124716557
Para  409  =  3.0056773219048383
Para  410  =  5.342687522794

Para  1004  =  3.5126766197066734
Para  1005  =  4.911910502621785
Para  1006  =  2.1259683727539276
Para  1007  =  3.077366591127069
Para  1008  =  4.3406590766051325
Para  1009  =  3.671629019746237
Para  1010  =  5.262450822849763
Para  1011  =  2.4664920642192576
Para  1012  =  5.801835395032098
Para  1013  =  5.323740148848109
Para  1014  =  5.485632694300527
Para  1015  =  4.936414251820693
Para  1016  =  7.5651927313180245
Para  1017  =  5.298136948870831
Para  1018  =  3.135082646612103
Para  1019  =  4.435029371420871
Para  1020  =  5.107515403192212
Para  1021  =  7.4707863004616835
Para  1022  =  3.021960322762246
Para  1023  =  5.71223446819351
Para  1024  =  5.2893591750694755
Para  1025  =  3.285397399601448
Para  1026  =  6.347655285450773
Para  1027  =  5.367753608291841
Para  1028  =  3.34321648328033
Para  1029  =  7.251047486287999
Para  1030  =  3.0283406958548076
Para  1031  =  5.639493551913325
Para  1032  =  4.1717886123314765
Para  1033  =  4.869810042726857
Par

Para  1254  =  2.1259683727539276
Para  1255  =  3.077366591127069
Para  1256  =  3.2751474213019165
Para  1257  =  4.445924992463386
Para  1258  =  3.1864655601050247
Para  1259  =  3.7623213845649386
Para  1260  =  6.366098606244218
Para  1261  =  2.8216662932835312
Para  1262  =  1.698030335923473
Para  1263  =  2.035124609613052
Para  1264  =  2.875894950280695
Para  1265  =  2.6311171791858077
Para  1266  =  3.0297215472803214
Para  1267  =  6.260516638806974
Para  1268  =  1.2282910127001514
Para  1269  =  2.1744524887366916
Para  1270  =  3.021960322762246
Para  1271  =  3.403392064534286
Para  1272  =  2.722629486515959
Para  1273  =  3.691731355167277
Para  1274  =  8.085666326021082
Para  1275  =  3.606887091728527
Para  1276  =  3.9777819692550525
Para  1277  =  7.5588950197752025
Para  1278  =  2.5137560117984434
Para  1279  =  5.940271348749207
Para  1280  =  7.54931364997025
Para  1281  =  2.4218468742174446
Para  1282  =  2.8280422004903842
Para  1283  =  7.9059726167635

Para  255  =  2.4218468742174446
Para  256  =  1.2282910127001514
Para  257  =  3.0297215472803214
Para  258  =  2.6311171791858077
Para  259  =  2.035124609613052
Para  260  =  1.698030335923473
Para  261  =  2.8216662932835312
Para  262  =  2.1259683727539276
Para  263  =  1.7192753113320216
Para  264  =  3.620854613701271
Para  265  =  2.135999715186046
Para  266  =  3.596650124716557
Para  267  =  3.0361307208864057
Para  268  =  3.5165484124817583
Para  269  =  2.4218468742174446
Para  270  =  2.4803916538860653
Para  271  =  4.0251201870271345
Para  272  =  2.6311171791858077
Para  273  =  2.8917656868737796
Para  274  =  1.698030335923473
Para  275  =  2.8216662932835312
Para  276  =  3.021417368817434
Para  277  =  1.7192753113320216
Para  278  =  3.620854613701271
Para  279  =  2.135999715186046
Para  280  =  2.6276285477387957
Para  281  =  3.614986210768792
Para  282  =  2.5129264803540248
Para  283  =  1.6343042849832385
Para  284  =  3.021960322762246
Para  285  =  2.17445

Para  755  =  5.483698921482173
Para  756  =  6.407623547454306
Para  757  =  6.9204371572832715
Para  758  =  4.891271357838791
Para  759  =  5.415455900420744
Para  760  =  6.4308862404740905
Para  761  =  6.257339677697275
Para  762  =  6.767759944225682
Para  763  =  4.869958393931508
Para  764  =  6.570523918726109
Para  765  =  3.4445803587183246
Para  766  =  7.2575162373439905
Para  767  =  4.573070676123101
Para  768  =  5.677058068375904
Para  769  =  8.998011342639861
Para  770  =  6.6580787698682276
Para  771  =  6.169082918632469
Para  772  =  7.170651714198943
Para  773  =  7.168458128670793
Para  774  =  4.832671418370021
Para  775  =  3.8640997452735864
Para  776  =  7.86598530579693
Para  777  =  5.3563398837825975
Para  778  =  4.443071781236279
Para  779  =  8.085214309241552
Para  780  =  6.188036422248113
Para  781  =  6.885869890291748
Para  782  =  6.582208769806089
Para  783  =  8.145468487996588
Para  784  =  6.2556610231735705
Para  785  =  5.1568193196802525


Para  1130  =  4.96387323025728
Para  1131  =  3.139917199413135
Para  1132  =  3.4880461203953823
Para  1133  =  3.3591414533578656
Para  1134  =  4.3700360660662785
Para  1135  =  3.22672952324774
Para  1136  =  2.228103976213526
Para  1137  =  6.23834277671205
Para  1138  =  8.964061655748903
Para  1139  =  5.429408796998825
Para  1140  =  3.215679174554097
Para  1141  =  8.510597431425033
Para  1142  =  4.449242084609313
Para  1143  =  2.781841690607453
Para  1144  =  5.342687522794339
Para  1145  =  3.696108414404421
Para  1146  =  3.0056773219048383
Para  1147  =  2.587638179332783
Para  1148  =  4.142778496874075
Para  1149  =  3.596650124716557
Para  1150  =  2.170460587795399
Para  1151  =  3.0942924536148784
Para  1152  =  3.0361307208864057
Para  1153  =  1.6343042849832385
Para  1154  =  3.5165484124817583
Para  1155  =  2.8280422004903842
Para  1156  =  2.4218468742174446
Para  1157  =  7.744833579871604
Para  1158  =  4.633742635854722
Para  1159  =  2.5137560117984434
Pa

### Uso da função para cálculo da distância entre itens e cálculo da distância entre dois pontos quaisquer da lista passada através do arquivo bloodPressureNishisatobook.D18.N15.txt usado na montagem do arquivo .csv

In [ ]:
Mdara = []
Mdcra = []
for linha in range(len(ra_antcd)):
    Mdara.append([])
    Mdcra.append([])
    aux1 = dist2ptsi(ra_antcd[linha], Mcides, Vfc, Des, rho, nlin)
    aux1 = ny.array(aux1, dtype=float32)
    Mdara[linha].append(aux1)
    aux2 = dist2ptsi(ra_consq[linha], Mcides, Vfc, Des, rho, nlin)
    aux2 = ny.array(aux2, dtype=float32)
    Mdcra[linha].append(aux2)

### Testando a nova meta imposta pelo mestre Altobelli (matrizes do antecedente e consequente)

In [ ]:
'''arq3 = open('itens_final_12-06-2019.csv', 'w', encoding='utf-8', newline='')
try:
    escreve = csv.writer(arq3, delimiter=';')
    escreve.writerow(['Antecedente', 'Consequente', 'Suporte', 'Confianca','Lift', 'Matriz Antecedentes',
                    'Matriz Consequentes', 'Pto Medio Anteced.', 'Pto Medio Conseq.', 'Suporte Anteced.',
                    'Suporte Conseq.', 'Dist. Anteced. -> Conseq.','Dist. Anteced. x Origem', 'Dist. Conseq. x Origem'])
    for i in range(len(confList)):
        b = ra_antcd[i], ra_consq[i], sup[i], confList[i][len(confList[i])-1],Lift[i], Mdara[i][0], Mdcra[i][0],pm_antcd[i],pm_consq[i],supant[i],supcons[i],distantcdconseq[i],dist_antecd_origem[i],dist_conseq_origem[i]
        escreve.writerow(b)
finally:
    arq3.close()'''
arq4 = open('itens_final_plot_12-06-2019.csv', 'w', encoding='utf-8', newline='')
try:
    escreve = csv.writer(arq4, delimiter=';')
    escreve.writerow(['Antcd', 'Consqt', 'Suporte', 'Confianca','Lift', 'Sup Antcd',
                      'D Antcd_0','Sup Consqt', 'D Conseq_0', 'D Antcd_D Consqt'])
    for i in range(len(confList)):
        b = ra_antcd[i], ra_consq[i], sup[i], confList[i][len(confList[i])-1], Lift[i], supant[i], dist_antecd_origem[i], supcons[i], dist_conseq_origem[i], distantcdconseq[i]
        escreve.writerow(b)
finally:
    arq4.close()

### Cálculo da distância entre os itens e as transações

In [ ]:
MDIeT = ny.zeros((len(Vfcc),len(Vfl)), dtype=float32)
for item in range(len(Vfcc)):
    for transacao in range(len(Vfl)):
        dist = 0
        for dimensao in range(Des):
            dist += rho[dimensao] * (((Mctd[transacao][dimensao])**2 / (Vfl[transacao] / ncol)) 
                    + ((Mcides[item][dimensao])**2 / (Vfcc[item] / nlin)) - 2 * sqrt(rho[dimensao]) 
                    * ((Mctd[transacao][dimensao]) * (Mcides[item][dimensao]) / sqrt((Vfl[transacao] / ncol) 
                    * (Vfcc[item] / nlin))))
        MDIeT[item, transacao] = dist
#pmatriz(MDIeT,'Matriz de distâncias entre Itens e Transações')

### Ponto médio da distância de todos os itens do grupo para cada uma das transações

In [ ]:
Vetpmit = ny.sum(MDIeT, axis=0)/MDIeT.shape[0]
#print(Vetpmit)